In [ ]:
# ✅ TELEGRAM CRYPTO BOT INTERAKTIF - V1
# Bot hanya aktif saat diperintah (/cek COIN TF), menganalisis menggunakan indikator teknikal

In [ ]:
!pip install python-telegram-bot pandas numpy

In [ ]:
import requests
import pandas as pd
import numpy as np
from telegram.ext import Updater, CommandHandler
from datetime import datetime
import logging

In [ ]:
# 🔐 GANTI DENGAN CREDENTIAL KAMU
OKX_API_KEY = 'ISI_API_KEY_KAMU'
OKX_API_SECRET = 'ISI_API_SECRET_KAMU'
OKX_API_PASSPHRASE = 'ISI_PASSPHRASE_KAMU'
TELEGRAM_TOKEN = 'ISI_TOKEN_BOT_KAMU'

In [ ]:
# ✅ Daftar coin wajib
COIN_LIST = [
    'BTC-USDT', 'ETH-USDT', 'SOL-USDT', 'XRP-USDT', 'TON-USDT',
    'DOGE-USDT', 'RENDER-USDT', 'TIA-USDT', 'AVAX-USDT', 'SUI-USDT',
    'LTC-USDT', 'UNI-USDT', 'PEPE-USDT', 'NEAR-USDT', 'ONDO-USDT'
]

In [ ]:
# Logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

In [ ]:
# Ambil data candle dari OKX
def get_candles(instId, bar='5m', limit=150):
    url = 'https://www.okx.com/api/v5/market/history-candles'
    params = {'instId': instId, 'bar': bar, 'limit': limit}
    response = requests.get(url, params=params)
    data = response.json()
    if data['code'] == '0':
        candles = data['data']
        df = pd.DataFrame(candles, columns=['ts','o','h','l','c','v','val'])
        df = df.iloc[::-1]
        df[['o','h','l','c','v']] = df[['o','h','l','c','v']].astype(float)
        df['ts'] = pd.to_datetime(df['ts'], unit='ms')
        df.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}, inplace=True)
        return df
    else:
        raise Exception(f"Gagal ambil data OKX: {data['msg']}")

In [ ]:
# Hitung indikator teknikal
def calculate_indicators(df):
    df['EMA_9'] = df['close'].ewm(span=9).mean()
    df['EMA_12'] = df['close'].ewm(span=12).mean()
    df['EMA_20'] = df['close'].ewm(span=20).mean()
    df['EMA_50'] = df['close'].ewm(span=50).mean()
    df['EMA_100'] = df['close'].ewm(span=100).mean()

In [ ]:
    ema12 = df['EMA_12']
    ema26 = df['close'].ewm(span=26).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']

In [ ]:
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df['RSI'] = 100 - (100 / (1 + rs))

In [ ]:
    df['MA20'] = df['close'].rolling(window=20).mean()
    df['BB_std'] = df['close'].rolling(window=20).std()
    df['BB_upper'] = df['MA20'] + 2 * df['BB_std']
    df['BB_lower'] = df['MA20'] - 2 * df['BB_std']

In [ ]:
    df['Vol_avg'] = df['volume'].rolling(window=20).mean()
    return df

In [ ]:
# Analisa sinyal sederhana
def analyze(df):
    latest = df.iloc[-1]
    signals = []

In [ ]:
    if latest['RSI'] < 30:
        signals.append('RSI: BUY (Oversold)')
    elif latest['RSI'] > 70:
        signals.append('RSI: SELL (Overbought)')

In [ ]:
    if latest['MACD'] > latest['MACD_signal']:
        signals.append('MACD: BUY')
    elif latest['MACD'] < latest['MACD_signal']:
        signals.append('MACD: SELL')

In [ ]:
    if latest['close'] > latest['EMA_9'] and latest['close'] > latest['EMA_12']:
        signals.append('EMA: BUY')
    elif latest['close'] < latest['EMA_9'] and latest['close'] < latest['EMA_12']:
        signals.append('EMA: SELL')

In [ ]:
    if latest['close'] >= latest['BB_upper']:
        signals.append('BB: SELL (Upper Band)')
    elif latest['close'] <= latest['BB_lower']:
        signals.append('BB: BUY (Lower Band)')

In [ ]:
    if latest['volume'] > 1.5 * latest['Vol_avg']:
        signals.append('Volume: Confirmed')

In [ ]:
    if not signals:
        signals.append('⛔ Tidak ada sinyal valid saat ini.')

In [ ]:
    return signals

In [ ]:
# Handler Telegram
def cek(update, context):
    try:
        if len(context.args) == 0:
            update.message.reply_text("Format: /cek COIN [TIMEFRAME]")
            return

In [ ]:
        coin = context.args[0].upper()
        tf = context.args[1] if len(context.args) > 1 else '5m'

In [ ]:
        if not coin.endswith("-USDT"):
            coin = coin.replace("USDT", "-USDT")

In [ ]:
        if coin not in COIN_LIST:
            update.message.reply_text("⚠️ Coin tidak ada di whitelist bot.")
            return

In [ ]:
        df = get_candles(coin, bar=tf)
        df = calculate_indicators(df)
        signals = analyze(df)
        latest = df.iloc[-1]

In [ ]:
        text = f"📊 *Analisa {coin} ({tf})*\n" \
             + f"Harga: {latest['close']:.4f}\n" \
             + f"RSI: {latest['RSI']:.2f} | MACD: {latest['MACD']:.4f}\n\n" \
             + '\n'.join(signals)

In [ ]:
        update.message.reply_text(text, parse_mode='Markdown')
    except Exception as e:
        update.message.reply_text(f"❌ Error: {str(e)}")

In [ ]:
# Run bot
updater = Updater(TELEGRAM_TOKEN)
dp = updater.dispatcher
dp.add_handler(CommandHandler("cek", cek))
updater.start_polling()
print("✅ Bot is running...")
